# Task 4: Loan Approval Prediction
## Binary Classification with Imbalanced Data Handling

**Objective:** Build a robust model to predict loan approval status

**Key Features:**
- Binary classification using Logistic Regression and Decision Tree
- Handle class imbalance with SMOTE
- Evaluate performance on imbalanced data
- Focus on precision, recall, and F1-score
- Model comparison and selection

**Dataset:** Loan-Approval-Prediction-Dataset (200+ loan applications)

## 1. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, roc_curve,
    precision_score, recall_score, f1_score, accuracy_score,
    precision_recall_curve, auc
)
from imblearn.over_sampling import SMOTE
import joblib
import os

print('✅ All libraries imported successfully!')
print(f'Working directory: {os.getcwd()}')

## 2. Load and Explore Dataset

In [ ]:
# Try to load from online source first, then local fallback
urls = [
    'https://raw.githubusercontent.com/siddharth-daga/Loan-Approval-Dataset/master/loan_data.csv',
    'https://kaggle.com/api/v1/datasets/download/altruistdream/loan-approval-prediction-dataset'
]

local_files = [
    'loan_data.csv',
    'loan_approval.csv',
    'Loan_Approval_Dataset.csv',
    '../Loan_Approval_Dataset.csv'
]

df = None
file_path = None

# Try online sources
for url in urls:
    try:
        print(f"Attempting to load from: {url}")
        df = pd.read_csv(url)
        file_path = url
        print(f"✅ Successfully loaded from online source!")
        break
    except Exception as e:
        print(f"❌ Failed: {str(e)[:50]}...")
        continue

# Try local files if online failed
if df is None:
    for file in local_files:
        if os.path.exists(file):
            try:
                print(f"\nAttempting to load from: {file}")
                df = pd.read_csv(file)
                file_path = file
                print(f"✅ Successfully loaded from local file!")
                break
            except Exception as e:
                print(f"❌ Failed: {str(e)[:50]}...")
                continue

if df is None:
    # Create a sample dataset if no file found
    print("\n⚠️ No dataset found. Creating sample loan data...")
    np.random.seed(42)
    n_samples = 300
    
    df = pd.DataFrame({
        'Loan_ID': range(1, n_samples + 1),
        'Age': np.random.randint(22, 70, n_samples),
        'Income': np.random.randint(20000, 150000, n_samples),
        'Credit_Score': np.random.randint(300, 850, n_samples),
        'Employment_Years': np.random.randint(0, 40, n_samples),
        'Loan_Amount': np.random.randint(5000, 500000, n_samples),
        'Gender': np.random.choice(['M', 'F'], n_samples),
        'Married': np.random.choice(['Yes', 'No'], n_samples),
        'Dependents': np.random.randint(0, 4, n_samples),
        'Education': np.random.choice(['Graduate', 'Not Graduate'], n_samples),
        'Self_Employed': np.random.choice(['Yes', 'No'], n_samples),
        'Approval': np.random.choice(['Approved', 'Rejected'], n_samples, p=[0.7, 0.3])
    })
    file_path = 'sample_loan_data'

print(f"\n{'='*60}")
print(f"Dataset: {file_path}")
print(f"Shape: {df.shape}")
print(f"{'='*60}")
print(f"\nFirst few rows:")
print(df.head())
print(f"\nColumn Information:")
print(df.info())

## 3. Data Preprocessing & Exploratory Data Analysis

In [ ]:
print("\n📊 MISSING VALUES ANALYSIS")
print("="*60)
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({'Missing_Count': missing, 'Percentage': missing_pct})
print(missing_df[missing_df['Missing_Count'] > 0])

# Handle missing values
for col in df.columns:
    if df[col].dtype == 'object':
        if df[col].isnull().sum() > 0:
            df[col].fillna(df[col].mode()[0], inplace=True)
    else:
        if df[col].isnull().sum() > 0:
            df[col].fillna(df[col].median(), inplace=True)

print(f"\n✅ Missing values handled!")

# Identify target and features
target_cols = ['Approval', 'Loan_Status', 'Status', 'Approved']
target_col = None
for col in target_cols:
    if col in df.columns:
        target_col = col
        break

if target_col is None:
    print("⚠️ Target column not found. Using last column as target.")
    target_col = df.columns[-1]

print(f"\nTarget Column: {target_col}")
print(f"Target Distribution:")
print(df[target_col].value_counts())
print(f"\nTarget Distribution (%):\n{df[target_col].value_counts(normalize=True) * 100}")

## 4. Feature Engineering & Encoding

In [ ]:
print("\n🔧 FEATURE ENGINEERING")
print("="*60)

# Separate features and target
X = df.drop(columns=[target_col])
y = df[target_col]

# Store column names for later
feature_columns = X.columns.tolist()

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include='object').columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"\nCategorical features ({len(categorical_cols)}): {categorical_cols}")
print(f"Numerical features ({len(numerical_cols)}): {numerical_cols}")

# Remove ID columns if present
id_cols = [col for col in numerical_cols if 'ID' in col or 'id' in col]
if id_cols:
    print(f"\nRemoving ID columns: {id_cols}")
    X = X.drop(columns=id_cols)
    numerical_cols = [col for col in numerical_cols if col not in id_cols]

# Encode target variable
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)
print(f"\nTarget classes: {le_target.classes_}")
print(f"Encoded: {np.unique(y_encoded)}")

# Encode categorical features
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le
    print(f"Encoded {col}: {dict(zip(le.classes_, le.transform(le.classes_)))}")

print(f"\n✅ Feature engineering complete!")
print(f"Final feature set shape: {X.shape}")
print(f"Target shape: {y_encoded.shape}")

## 5. Class Imbalance Analysis & Visualization

In [ ]:
print("\n📊 CLASS IMBALANCE ANALYSIS")
print("="*60)

# Calculate class distribution
unique, counts = np.unique(y_encoded, return_counts=True)
class_distribution = dict(zip(le_target.classes_, counts))

print(f"\nClass Distribution:")
for cls, count in class_distribution.items():
    pct = (count / len(y_encoded)) * 100
    print(f"  {cls}: {count} ({pct:.2f}%)")

imbalance_ratio = counts.max() / counts.min()
print(f"\nImbalance Ratio: {imbalance_ratio:.2f}:1")

# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot
axes[0].bar(le_target.classes_, counts, color=['#FF6B6B', '#4ECDC4'])
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_title('Loan Approval Distribution (Before SMOTE)', fontsize=13, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Pie chart
axes[1].pie(counts, labels=le_target.classes_, autopct='%1.1f%%', 
            colors=['#FF6B6B', '#4ECDC4'], startangle=90)
axes[1].set_title('Class Distribution %', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.savefig('outputs/01_class_distribution.png', dpi=300, bbox_inches='tight')
print("\n✅ Saved: outputs/01_class_distribution.png")
plt.show()

## 6. Train-Test Split & Feature Scaling

In [ ]:
print("\n🔀 TRAIN-TEST SPLIT")
print("="*60)

# Split data (80-20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"\nTraining set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")

print(f"\nTraining set class distribution:")
unique_train, counts_train = np.unique(y_train, return_counts=True)
for cls, count in zip(le_target.classes_, counts_train):
    pct = (count / len(y_train)) * 100
    print(f"  {cls}: {count} ({pct:.2f}%)")

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n✅ Data scaled using StandardScaler")
print(f"Scaled training data shape: {X_train_scaled.shape}")

## 7. Apply SMOTE to Handle Class Imbalance (BONUS)

In [ ]:
print("\n⚖️ APPLYING SMOTE (Synthetic Minority Over-sampling Technique)")
print("="*60)

# Apply SMOTE to training data only
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

print(f"\nOriginal training set shape: {X_train_scaled.shape}")
print(f"SMOTE training set shape: {X_train_smote.shape}")

print(f"\nOriginal class distribution (training):")
unique_orig, counts_orig = np.unique(y_train, return_counts=True)
for cls, count in zip(le_target.classes_, counts_orig):
    pct = (count / len(y_train)) * 100
    print(f"  {cls}: {count} ({pct:.2f}%)")

print(f"\nAfter SMOTE class distribution (training):")
unique_smote, counts_smote = np.unique(y_train_smote, return_counts=True)
for cls, count in zip(le_target.classes_, counts_smote):
    pct = (count / len(y_train_smote)) * 100
    print(f"  {cls}: {count} ({pct:.2f}%)")

# Visualize SMOTE effect
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].bar(le_target.classes_, counts_orig, color=['#FF6B6B', '#4ECDC4'])
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_title('Class Distribution Before SMOTE', fontsize=13, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

axes[1].bar(le_target.classes_, counts_smote, color=['#FF6B6B', '#4ECDC4'])
axes[1].set_ylabel('Count', fontsize=12)
axes[1].set_title('Class Distribution After SMOTE', fontsize=13, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/02_smote_comparison.png', dpi=300, bbox_inches='tight')
print("\n✅ Saved: outputs/02_smote_comparison.png")
plt.show()

## 8. Model Training: Logistic Regression

In [ ]:
print("\n🤖 TRAINING LOGISTIC REGRESSION MODEL")
print("="*60)

# Train Logistic Regression on SMOTE data
lr_model = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')
lr_model.fit(X_train_smote, y_train_smote)

# Predictions
y_pred_lr = lr_model.predict(X_test_scaled)
y_pred_proba_lr = lr_model.predict_proba(X_test_scaled)[:, 1]

# Cross-validation
cv_scores_lr = cross_val_score(lr_model, X_train_smote, y_train_smote, cv=5, scoring='f1')

print(f"\n✅ Logistic Regression trained successfully!")
print(f"Cross-validation F1 scores: {cv_scores_lr}")
print(f"Mean CV F1-Score: {cv_scores_lr.mean():.4f} (+/- {cv_scores_lr.std():.4f})")

## 9. Model Training: Decision Tree (BONUS)

In [ ]:
print("\n🌳 TRAINING DECISION TREE MODEL (BONUS)")
print("="*60)

# Train Decision Tree on SMOTE data
dt_model = DecisionTreeClassifier(
    max_depth=10, 
    random_state=42, 
    class_weight='balanced',
    min_samples_split=5,
    min_samples_leaf=2
)
dt_model.fit(X_train_smote, y_train_smote)

# Predictions
y_pred_dt = dt_model.predict(X_test_scaled)
y_pred_proba_dt = dt_model.predict_proba(X_test_scaled)[:, 1]

# Cross-validation
cv_scores_dt = cross_val_score(dt_model, X_train_smote, y_train_smote, cv=5, scoring='f1')

print(f"\n✅ Decision Tree trained successfully!")
print(f"Cross-validation F1 scores: {cv_scores_dt}")
print(f"Mean CV F1-Score: {cv_scores_dt.mean():.4f} (+/- {cv_scores_dt.std():.4f})")

# Feature importance
feature_importance_dt = pd.DataFrame({
    'Feature': X.columns,
    'Importance': dt_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\nTop 10 Most Important Features:")
print(feature_importance_dt.head(10))

## 10. Comprehensive Model Evaluation (Focus on Imbalanced Data Metrics)

In [ ]:
print("\n📊 MODEL EVALUATION ON IMBALANCED DATA")
print("="*60)

# Logistic Regression Evaluation
print("\n" + "="*60)
print("LOGISTIC REGRESSION PERFORMANCE")
print("="*60)

print(f"\nAccuracy: {accuracy_score(y_test, y_pred_lr):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_lr):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_lr):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_lr):.4f}")
print(f"ROC-AUC Score: {roc_auc_score(y_test, y_pred_proba_lr):.4f}")

print(f"\nClassification Report (Logistic Regression):")
print(classification_report(y_test, y_pred_lr, target_names=le_target.classes_))

# Decision Tree Evaluation
print("\n" + "="*60)
print("DECISION TREE PERFORMANCE")
print("="*60)

print(f"\nAccuracy: {accuracy_score(y_test, y_pred_dt):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_dt):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_dt):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_dt):.4f}")
print(f"ROC-AUC Score: {roc_auc_score(y_test, y_pred_proba_dt):.4f}")

print(f"\nClassification Report (Decision Tree):")
print(classification_report(y_test, y_pred_dt, target_names=le_target.classes_))

## 11. Confusion Matrices & ROC Curves

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Confusion Matrix - Logistic Regression
cm_lr = confusion_matrix(y_test, y_pred_lr)
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', ax=axes[0, 0], 
            xticklabels=le_target.classes_, yticklabels=le_target.classes_)
axes[0, 0].set_title('Confusion Matrix - Logistic Regression', fontsize=13, fontweight='bold')
axes[0, 0].set_ylabel('True Label')
axes[0, 0].set_xlabel('Predicted Label')

# Confusion Matrix - Decision Tree
cm_dt = confusion_matrix(y_test, y_pred_dt)
sns.heatmap(cm_dt, annot=True, fmt='d', cmap='Greens', ax=axes[0, 1],
            xticklabels=le_target.classes_, yticklabels=le_target.classes_)
axes[0, 1].set_title('Confusion Matrix - Decision Tree', fontsize=13, fontweight='bold')
axes[0, 1].set_ylabel('True Label')
axes[0, 1].set_xlabel('Predicted Label')

# ROC Curves
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_pred_proba_lr)
fpr_dt, tpr_dt, _ = roc_curve(y_test, y_pred_proba_dt)

auc_lr = roc_auc_score(y_test, y_pred_proba_lr)
auc_dt = roc_auc_score(y_test, y_pred_proba_dt)

axes[1, 0].plot(fpr_lr, tpr_lr, label=f'Logistic Regression (AUC={auc_lr:.3f})', linewidth=2)
axes[1, 0].plot(fpr_dt, tpr_dt, label=f'Decision Tree (AUC={auc_dt:.3f})', linewidth=2)
axes[1, 0].plot([0, 1], [0, 1], 'k--', label='Random Classifier')
axes[1, 0].set_xlabel('False Positive Rate')
axes[1, 0].set_ylabel('True Positive Rate')
axes[1, 0].set_title('ROC Curves Comparison', fontsize=13, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# Precision-Recall Curves
precision_lr, recall_lr, _ = precision_recall_curve(y_test, y_pred_proba_lr)
precision_dt, recall_dt, _ = precision_recall_curve(y_test, y_pred_proba_dt)

pr_auc_lr = auc(recall_lr, precision_lr)
pr_auc_dt = auc(recall_dt, precision_dt)

axes[1, 1].plot(recall_lr, precision_lr, label=f'Logistic Regression (AUC={pr_auc_lr:.3f})', linewidth=2)
axes[1, 1].plot(recall_dt, precision_dt, label=f'Decision Tree (AUC={pr_auc_dt:.3f})', linewidth=2)
axes[1, 1].set_xlabel('Recall')
axes[1, 1].set_ylabel('Precision')
axes[1, 1].set_title('Precision-Recall Curves', fontsize=13, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/03_model_evaluation.png', dpi=300, bbox_inches='tight')
print("✅ Saved: outputs/03_model_evaluation.png")
plt.show()

## 12. Model Comparison & Selection

In [ ]:
print("\n🏆 MODEL COMPARISON")
print("="*60)

comparison_df = pd.DataFrame({
    'Model': ['Logistic Regression', 'Decision Tree'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_lr),
        accuracy_score(y_test, y_pred_dt)
    ],
    'Precision': [
        precision_score(y_test, y_pred_lr),
        precision_score(y_test, y_pred_dt)
    ],
    'Recall': [
        recall_score(y_test, y_pred_lr),
        recall_score(y_test, y_pred_dt)
    ],
    'F1-Score': [
        f1_score(y_test, y_pred_lr),
        f1_score(y_test, y_pred_dt)
    ],
    'ROC-AUC': [
        roc_auc_score(y_test, y_pred_proba_lr),
        roc_auc_score(y_test, y_pred_proba_dt)
    ]
})

print("\n" + comparison_df.to_string(index=False))

# Select best model based on F1-score
best_model_idx = comparison_df['F1-Score'].idxmax()
best_model_name = comparison_df.loc[best_model_idx, 'Model']
best_f1 = comparison_df.loc[best_model_idx, 'F1-Score']

print(f"\n🏆 Best Model: {best_model_name} (F1-Score: {best_f1:.4f})")

# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics = ['Precision', 'Recall', 'F1-Score']
colors = ['#FF6B6B', '#4ECDC4']

for idx, metric in enumerate(metrics):
    values = comparison_df[metric].values
    axes[idx].bar(comparison_df['Model'], values, color=colors)
    axes[idx].set_ylabel(metric, fontsize=12)
    axes[idx].set_title(f'{metric} Comparison', fontsize=13, fontweight='bold')
    axes[idx].set_ylim([0, 1])
    axes[idx].grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for i, v in enumerate(values):
        axes[idx].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('outputs/04_model_comparison.png', dpi=300, bbox_inches='tight')
print("\n✅ Saved: outputs/04_model_comparison.png")
plt.show()

## 13. Feature Importance Analysis (BONUS)

In [ ]:
print("\n🔍 FEATURE IMPORTANCE ANALYSIS")
print("="*60)

# Decision Tree Feature Importance
feature_importance_dt = pd.DataFrame({
    'Feature': X.columns,
    'Importance': dt_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nDecision Tree - Top 10 Features:")
print(feature_importance_dt.head(10))

# Logistic Regression Coefficients
feature_importance_lr = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': np.abs(lr_model.coef_[0])
}).sort_values('Coefficient', ascending=False)

print("\nLogistic Regression - Top 10 Features (by coefficient magnitude):")
print(feature_importance_lr.head(10))

# Visualize top features
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Decision Tree
top_dt = feature_importance_dt.head(10)
axes[0].barh(top_dt['Feature'], top_dt['Importance'], color='#4ECDC4')
axes[0].set_xlabel('Importance', fontsize=12)
axes[0].set_title('Decision Tree - Top 10 Important Features', fontsize=13, fontweight='bold')
axes[0].invert_yaxis()

# Logistic Regression
top_lr = feature_importance_lr.head(10)
axes[1].barh(top_lr['Feature'], top_lr['Coefficient'], color='#FF6B6B')
axes[1].set_xlabel('Coefficient Magnitude', fontsize=12)
axes[1].set_title('Logistic Regression - Top 10 Important Features', fontsize=13, fontweight='bold')
axes[1].invert_yaxis()

plt.tight_layout()
plt.savefig('outputs/05_feature_importance.png', dpi=300, bbox_inches='tight')
print("\n✅ Saved: outputs/05_feature_importance.png")
plt.show()

## 14. Save Trained Models

In [ ]:
print("\n💾 SAVING MODELS")
print("="*60)

# Create model directory if it doesn't exist
os.makedirs('model', exist_ok=True)

# Save models
joblib.dump(lr_model, 'model/logistic_regression_model.pkl')
joblib.dump(dt_model, 'model/decision_tree_model.pkl')
joblib.dump(scaler, 'model/scaler.pkl')
joblib.dump(label_encoders, 'model/label_encoders.pkl')
joblib.dump(le_target, 'model/target_encoder.pkl')

print("\n✅ Models saved:")
print("   - model/logistic_regression_model.pkl")
print("   - model/decision_tree_model.pkl")
print("   - model/scaler.pkl")
print("   - model/label_encoders.pkl")
print("   - model/target_encoder.pkl")

# Save feature columns for later use
joblib.dump(X.columns.tolist(), 'model/feature_columns.pkl')
print("   - model/feature_columns.pkl")

## 15. Test Predictions on New Data

In [ ]:
print("\n🔮 TESTING PREDICTIONS ON NEW DATA")
print("="*60)

# Create sample new customers for prediction
new_customers = pd.DataFrame({
    'Age': [35, 45, 28],
    'Income': [50000, 120000, 35000],
    'Credit_Score': [720, 800, 650],
    'Employment_Years': [5, 15, 2],
    'Loan_Amount': [100000, 300000, 50000],
    'Gender': ['M', 'F', 'M'],
    'Married': ['Yes', 'Yes', 'No'],
    'Dependents': [1, 2, 0],
    'Education': ['Graduate', 'Graduate', 'Not Graduate'],
    'Self_Employed': ['No', 'No', 'Yes']
})

# Make predictions if all required columns exist
try:
    # Encode categorical features
    new_customers_encoded = new_customers.copy()
    for col in categorical_cols:
        if col in new_customers_encoded.columns:
            new_customers_encoded[col] = label_encoders[col].transform(new_customers_encoded[col].astype(str))
    
    # Scale features
    new_customers_scaled = scaler.transform(new_customers_encoded[X.columns])
    
    # Predictions
    pred_lr = lr_model.predict(new_customers_scaled)
    pred_proba_lr = lr_model.predict_proba(new_customers_scaled)
    
    pred_dt = dt_model.predict(new_customers_scaled)
    pred_proba_dt = dt_model.predict_proba(new_customers_scaled)
    
    print("\nSample New Customers:")
    print(new_customers.to_string())
    
    print("\n" + "="*60)
    print("PREDICTIONS")
    print("="*60)
    
    for i in range(len(new_customers)):
        print(f"\nCustomer {i+1}:")
        print(f"  Age: {new_customers.iloc[i]['Age']}, Income: ${new_customers.iloc[i]['Income']:,}")
        print(f"  Credit Score: {new_customers.iloc[i]['Credit_Score']}, Employment: {new_customers.iloc[i]['Employment_Years']} years")
        
        print(f"  \n  Logistic Regression:")
        print(f"    Prediction: {le_target.classes_[pred_lr[i]]}")
        print(f"    Confidence: {pred_proba_lr[i].max():.2%}")
        
        print(f"  \n  Decision Tree:")
        print(f"    Prediction: {le_target.classes_[pred_dt[i]]}")
        print(f"    Confidence: {pred_proba_dt[i].max():.2%}")

except Exception as e:
    print(f"⚠️ Could not create sample predictions: {e}")
    print("This is expected if categorical columns don't match the training data exactly.")

## 16. Summary Report

In [ ]:
print("\n" + "="*60)
print("📋 TASK 4 EXECUTION SUMMARY")
print("="*60)

summary = f"""
LOAN APPROVAL PREDICTION - BINARY CLASSIFICATION
{'='*60}

📊 DATASET INFORMATION:
   Total Samples: {len(df)}
   Features: {len(feature_columns)}
   Target: {target_col}
   Classes: {', '.join(le_target.classes_)}
   Class Imbalance Ratio: {imbalance_ratio:.2f}:1

🔄 DATA PREPROCESSING:
   Missing Values: Handled with mode/median imputation
   Categorical Features: {len(categorical_cols)} features encoded
   Feature Scaling: StandardScaler applied
   SMOTE Applied: YES (balanced training data)

🤖 MODELS TRAINED:
   1. Logistic Regression
      - Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}
      - Precision: {precision_score(y_test, y_pred_lr):.4f}
      - Recall: {recall_score(y_test, y_pred_lr):.4f}
      - F1-Score: {f1_score(y_test, y_pred_lr):.4f}
      - ROC-AUC: {roc_auc_score(y_test, y_pred_proba_lr):.4f}

   2. Decision Tree (BONUS)
      - Accuracy: {accuracy_score(y_test, y_pred_dt):.4f}
      - Precision: {precision_score(y_test, y_pred_dt):.4f}
      - Recall: {recall_score(y_test, y_pred_dt):.4f}
      - F1-Score: {f1_score(y_test, y_pred_dt):.4f}
      - ROC-AUC: {roc_auc_score(y_test, y_pred_proba_dt):.4f}

🏆 BEST MODEL: {best_model_name}
   F1-Score: {best_f1:.4f}

📁 FILES GENERATED:
   Models:
      - model/logistic_regression_model.pkl
      - model/decision_tree_model.pkl
      - model/scaler.pkl
      - model/label_encoders.pkl
      - model/target_encoder.pkl
      - model/feature_columns.pkl

   Visualizations:
      - outputs/01_class_distribution.png
      - outputs/02_smote_comparison.png
      - outputs/03_model_evaluation.png
      - outputs/04_model_comparison.png
      - outputs/05_feature_importance.png

✅ TASK COMPLETION: 100%
   ✓ EDA completed
   ✓ Data preprocessing done
   ✓ Class imbalance handled with SMOTE
   ✓ Two models trained and compared
   ✓ Comprehensive evaluation on imbalanced data
   ✓ Feature importance analyzed
   ✓ Models saved for deployment

🚀 NEXT STEP: Build Streamlit app for interactive predictions
"""

print(summary)

# Save summary to file
with open('outputs/summary.txt', 'w') as f:
    f.write(summary)

print("\n✅ Summary saved to: outputs/summary.txt")